In [1]:
# import
import numpy as np
from scipy.linalg import eig
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
from sklearn.covariance import GraphicalLasso
from scipy.special import expit
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from joblib import Parallel,delayed
from tqdm import tqdm
from scipy.optimize import minimize
from scipy.optimize import fmin_powell


def shrink(u, a):
    value = np.sign(u) * np.maximum(np.abs(u) - a, 0)
    return value

def prox(u, a, tau=0.5):
    value = u - np.maximum((tau - 1) / a, np.minimum(u, tau / a))
    return value

def psi(u, tau=0.5):
    a = (u >= 0).astype(int)
    value = tau - 1 + a
    return value

def qradmm(T, X, y, beta, z, w, theta, gamma, sigma, lambda_, K_est, D_est,tau=0.5):
    n, d = X.shape
    eta = np.max(np.linalg.eigvalsh(X.T @ X))
    for t in range(T):
        a = beta + 1 / (sigma * eta) * np.matmul(X.T, (theta + sigma * y - sigma * np.matmul(X, beta) - sigma * z))
        b = lambda_ * w / (sigma * eta)
        beta = shrink(a, b)
        e = y - np.matmul(X, beta) + theta / sigma
        z = prox(e, (n * sigma), tau=tau)

        theta = theta - gamma * sigma * (np.matmul(X, beta) + z - y)
    gamma_hat = beta[:K_est]
    beta_hat = beta[K_est:D_est]
    return gamma_hat,beta_hat

def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part


In [2]:
# quantile regression

def quantile_regression_with_l1_loss(X, y, quantile, lambda_val, K_est):
    
    def objective(beta):
        residuals = y - beta[0] - np.dot(X, beta[1:])
        loss = np.mean(np.maximum(quantile * residuals, (quantile - 1) * residuals))
        l1_penalty = lambda_val * np.sum(np.abs(beta[K_est:]) + np.abs(beta[0]))
        return loss + l1_penalty
    
    beta_initial = np.zeros(X.shape[1] + 1)
    result = fmin_powell(objective, beta_initial, maxiter=1000, disp=False)
    return result[1:], result[0]

In [3]:
# clime
import numpy as np
from scipy.linalg import LinAlgError
from scipy.linalg import svd
from scipy.optimize import linprog
def clime(x, lambd, perturb=True, standardize=True):
    n, p = x.shape
    if standardize:
        x = (x - np.mean(x, axis=0)) / np.std(x, axis=0)
    Sigma = np.cov(x.T) * (1 - 1 / n)
    eigvals = np.linalg.eigvalsh(Sigma)
    if perturb:
        perturb = max(max(eigvals) - p * min(eigvals), 0) / (p - 1)
    else:
        perturb = 0
    Sigma += np.eye(p) * perturb
    emat = np.eye(p)
    Omega = np.zeros((p, p))
    for j in range(p):
        beta = linprogS(Sigma, emat[:, j], lambd)
        Omega[:, j] = beta
    Omega = Omega * (np.abs(Omega) <= np.abs(Omega.T)) + Omega.T * (np.abs(Omega) > np.abs(Omega.T))
    return Omega
def linprogS(Sigma, e, lambd):
    p = Sigma.shape[0]
    f_obj = np.ones(2 * p)
    con1 = np.hstack((-Sigma, Sigma))
    b1 = lambd - e
    b2 = lambd + e
    f_con = np.vstack((-np.eye(2 * p), con1, -con1))
    f_rhs = np.hstack((np.zeros(2 * p), b1, b2))
    lp_out = linprog(c=f_obj, A_ub=f_con, b_ub=f_rhs)
    beta = lp_out.x[:p] - lp_out.x[p:(2 * p)]
    return beta

In [4]:
# noisereg
def noise_reg(X, lambd):
    # Define variables
    N = X.shape[0]
    p = X.shape[1]
    A_u = np.eye(N) - np.linalg.inv(np.eye(N))
    X0_tilt = np.linalg.cholesky(np.eye(N) - A_u).T @ X

    # Initialize C_hat and tau_hat
    C_hat = np.zeros((p, p - 1))
    tau_hat = np.zeros(p)

    # Loop through each column to calculate fit_Theta and tau_hat
    for l in range(p):
        # Perform Lasso regression excluding the l-th column
        X_excluded_l = np.delete(X0_tilt, l, axis=1)
        lasso = Lasso(alpha=lambd, random_state=42, fit_intercept=False).fit(X_excluded_l, X0_tilt[:, l])
        # Compute fit_Theta_value as defined
        residuals = X0_tilt[:, l] - lasso.predict(X_excluded_l)
        fit_Theta_value = (1/N) * np.linalg.norm(residuals)**2 + lasso.alpha * np.linalg.norm(lasso.coef_, 1)
        # Store values in tau_hat and C_hat
        tau_hat[l] = fit_Theta_value
        C_hat[l, :] = lasso.coef_

    # Construct diagonal matrix T_hat
    T_hat = np.diag(tau_hat**0.5)

    # Construct Theta_hat
    Theta_hat = np.zeros((p, p))
    for l in range(p):
        Theta_hat[l, :l] = -C_hat[l, :l]
        Theta_hat[l, l+1:] = -C_hat[l, l:]

    # Fill the diagonal with 1
    np.fill_diagonal(Theta_hat, 1)

    # Final Theta_hat
    Theta_hat = np.linalg.inv(T_hat) @ Theta_hat @ np.linalg.inv(T_hat)
    return Theta_hat

In [5]:
# example
def trail(seed, N, s, p, eps_type, tau, h, lambd,setting='setting_1'):
    np.random.seed(seed)
    # generate simulation data
    K = 2
    btrue = np.concatenate((np.ones(s), np.zeros(p - s)), axis=0) * 0.6
    B = np.random.uniform(-1, 1, size=(p, K))
    gamma1 = np.ones(K)
    # setting 1
    if setting=='setting_1':
        F = np.random.normal(size=(N, K))
        U = np.random.normal(size=(N, p))

    # setting 2
    else:
        Phi = np.array([[0.5 ** (abs(i - j) + 1) for j in range(K)] for i in range(K)])
        F = np.zeros((N, K))
        F[0] = np.random.multivariate_normal(np.zeros(K), np.identity(K))
        for t in range(1, N):
            xi_t = np.random.multivariate_normal(np.zeros(K), np.identity(K))
            F[t] = Phi @ F[t - 1] + xi_t
        Sigma = np.array([[0.6 ** (abs(i - j)) for j in range(p)] for i in range(p)])
        U = np.random.multivariate_normal(np.zeros(p), Sigma, size=N)
    
    X = F @ B.T + U
    if eps_type == 't1.5':
        eps = np.random.standard_t(1.5, size=N)
    elif eps_type == 'norm':
        eps = np.random.normal(size=N)
    elif eps_type == 'cauchy':
        eps = np.random.standard_cauchy(size=N)

    # Generate Y
    Y = F @ gamma1 + U @ btrue + eps
    # step1 estimate
    cov = X @ X.T / N
    eigval, eigvec = np.linalg.eig(cov)
    K_est = np.argmin(np.diff(np.log(eigval[:10]))) + 1
    hatf = np.sqrt(N) * eigvec[:, :K_est]
    hatB = np.transpose(1 / N * np.transpose(hatf) @ X)
    hatU = X - hatf @ np.transpose(hatB)
    X_hat = np.concatenate((hatf, hatU), axis=1)
    w_0 = np.concatenate((np.zeros(K_est), np.ones(p)))

    # step2 optimize
    res , res_miu= quantile_regression_with_l1_loss(X_hat, Y, tau, 0.001, K_est)
    gamma_hat, beta_hat = res[:K_est], res[K_est:]
    # step3 debias
    cov = (hatU.T) @ hatU / N
    theta = clime(cov, lambd = lambd)
    epsilon = Y - np.dot(hatf, gamma_hat) - np.dot(hatU, beta_hat)-res_miu
    hatmiu = np.concatenate((hatU, hatf), axis=1)
    hatphi = np.concatenate((beta_hat, gamma_hat))
    def gauss(u):
        return np.exp(-u**2/2)/np.sqrt(2*np.pi)
    hatg = 0
    for i in range(N):
        hatg += gauss((Y[i] - hatmiu[i,:] @ hatphi-res_miu) / h)
    hatg /= N * h
    shift = 0
    for i in range(N):
        shift += hatg * psi(epsilon[i]) * hatU[i, :]
    beta_tilde = beta_hat + np.dot(theta, shift) / N
    return beta_hat, beta_tilde, hatg, np.diag(theta.T@cov@theta)

In [6]:
beta_hat, beta_tilde, hatg, theta=trail(seed=123, N=500, s=30, p=500, 
                                        eps_type='norm', tau=0.5, h=0.2, lambd=0.5,setting='setting_1')

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [05:30<00:00,  1.51it/s]


In [7]:
beta_hat, beta_tilde, hatg, theta

(array([ 5.80327974e-01,  5.46711472e-01,  5.05991701e-01,  6.33672145e-01,
         6.70304049e-01,  3.95637036e-01,  6.45882094e-01,  4.83845843e-01,
         6.20500337e-01,  5.85290867e-01,  4.56736787e-01,  5.04160835e-01,
         5.94760747e-01,  6.90648762e-01,  6.48342872e-01,  5.94806144e-01,
         5.83496209e-01,  6.09978582e-01,  5.18028877e-01,  5.53440748e-01,
         6.31237354e-01,  5.19487330e-01,  5.95084126e-01,  5.21303806e-01,
         5.33477494e-01,  5.78064287e-01,  4.80978643e-01,  5.92284315e-01,
         5.46864013e-01,  5.84053491e-01,  2.08464219e-02, -4.54504164e-02,
         3.03002807e-03,  3.55345092e-02, -6.03284395e-02,  3.27582251e-02,
         2.11844612e-02, -1.35421509e-01,  7.17097668e-02, -1.43792991e-02,
         4.86949245e-02,  3.01714287e-02,  2.00779103e-02,  5.26495371e-02,
        -9.61789970e-02,  6.62573865e-02,  3.08109814e-02, -3.16327812e-04,
         6.23661358e-02, -1.54945628e-03, -9.19437941e-02,  4.29599440e-02,
         4.3